<center>

**Vježba 2 - Osnovni IRIS model i MLflow**

</center>

U ovoj vježbi želimo pokazati osnovno korištenje *MLflow* servera na IRIS skupu podataka.

Prvo učitavamo potrebne python module.

In [ ]:
from sklearn import datasets
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
import pickle

import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.client import MlflowClient
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository

Prvo učitamo IRIS skup podataka iz *scikit-learn* modula:
- Učitamo skup podataka, te ga razdvojimo na ulaze i labele
- Na ulaznom skupu naučimo *scaler* koji nam služi za transformiranje ulaza
- Spremimo *scaler* model u *pickle* format (mogli smo i u *joblib*)
- Razdvojimo IRIS skup podataka na podskup za učenje i testiranje

In [ ]:
def loadIrisDataset():
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
    return (X,y)

(X,y)=loadIrisDataset()
display(X)

scaler = StandardScaler()

X_scal = scaler.fit_transform(X)
X_scal = pd.DataFrame(X_scal)

pickle.dump(scaler, open('scaler.pkl', 'wb')) 

X_train, X_test, y_train, y_test = train_test_split(X_scal, y, test_size = 0.2)

Postavimo *MLflow* server URL. Zatim potražimo naš IRIS eksperiment.  
U slučaju da eksperiment nije pronađen, stvorimo ga.

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
exps = mlflow.search_experiments(filter_string="name LIKE '%Iris%'")
if len(exps)==0:
    exp_id = mlflow.create_experiment("MLP - Iris dataset")
else:
    exp_id = exps[0].experiment_id

Prvo naučimo MLP klasifikator na podskupu za učenje.  
Zatim detektiramo sučelje MLP klasifikatora na temelju podskupa za učenje.  

Svaki poziv *MLflow-a* funkcionira kao posebno *pokretanje - run*. U našem slučaju smo pokretanju dali posebni naziv s kojim uočavamo da se radi o učenju modela.  
U našem pokretanju zapišemo model koji smo prethodno naučili. Dodatno zapišemo i *scaler* za transformaciju ulaznih podataka.  
Posebno definiramo meta-podatke o modelu koji smo upisali.

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=2000, verbose=1)
mlp.fit(X_train, y_train)

schema = infer_signature(X_train, y_train)
with mlflow.start_run(run_name="Training iris MLP model",
                        experiment_id=exp_id) as run:
    mlflow.sklearn.log_model(mlp, artifact_path="sklearn-model", signature=schema)
    mlflow.log_artifact('scaler.pkl', artifact_path="sklearn-model") 	
        
name="Iris MLP model"

client = MlflowClient()
rms = client.search_registered_models("name='"+name+"'")
if len(rms)==0: client.create_registered_model(name)

model_uri = "runs:/{}/sklearn-model".format(run.info.run_id)
model_src = RunsArtifactRepository.get_underlying_uri(model_uri)
mv = client.create_model_version(name, model_src, run.info.run_id, 
    description="MLP model for iris dataset")

mlflow.end_run()

Zatim taj naučeni model automatski evaluiramo na *MLflow* serveru.  
Tu evaluaciju definiramo kao posebno pokretanje specifičnog naziva.  
Oba pokretanja sada možemo provjeriti na korisničkom sučelju *MLflow* servera.

In [ ]:
client = MlflowClient()
models = client.get_latest_versions(name, stages=["None"])
eval_data = X_test
eval_data["target"] = y_test

with mlflow.start_run(run_name="Testing iris MLP model, version:"+models[0].version,
                        experiment_id=exp_id) as run:
    mlflow.evaluate(models[0].source,eval_data,targets="target",model_type="classifier",
        evaluators=["default"])

mlflow.end_run()